In [1]:
import pandas as pd
import numpy as np
import json
import os
import requests
from bs4 import BeautifulSoup

In [6]:
raptor_df = pd.read_csv("modern_RAPTOR_by_team.csv")

In [8]:
#Clean Data

# Specify the columns you want to keep
columns_to_keep = ["player_name","season","season_type","team","poss","mp","raptor_offense","raptor_total",
                   "war_total","war_reg_season","war_playoffs","predator_offense","predator_total"]

# Keep only the specified columns
raptor_df = raptor_df[columns_to_keep]

raptor_df.head()

,player_name,season,season_type,team,poss,mp,raptor_offense,raptor_total,war_total,war_reg_season,war_playoffs,predator_offense,predator_total
0,Alex Abrines,2017,PO,OKC,172,80,-0.892617,-7.453875,-0.198700,0.000000,-0.198700,-3.298178,-9.833292
1,Alex Abrines,2017,RS,OKC,2215,1055,0.654933,-0.069300,1.447708,1.447708,0.000000,0.339201,-0.272665
2,Alex Abrines,2018,PO,OKC,233,110,1.875157,2.615450,0.311392,0.000000,0.311392,2.877519,2.356566
3,Alex Abrines,2018,RS,OKC,2313,1134,-0.211818,-1.940401,0.465912,0.465912,0.000000,-0.482078,-1.654306
4,Alex Abrines,2019,RS,OKC,1279,588,-4.040157,-2.154538,0.178167,0.178167,0.000000,-4.577678,-3.034396
